In [1]:
! pip install pytesseract
! pip install pdf2image
! apt-get install poppler-utils
! apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 32 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 0s (1,782 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121747 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.3_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.3) ...
Setting up poppler-utils (22.02.0-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd

# PDF to text

In [3]:
import io
import pytesseract
from pdf2image import convert_from_path

def extract_text_from_pdf(pdf_path):
    pages = convert_from_path(pdf_path, 500)

    # Extract text from each page using Tesseract OCR
    text_data = ''
    for page in pages:
        text = pytesseract.image_to_string(page)
        text_data += text + '\n'

    return text_data

extracted_text = extract_text_from_pdf('path_pdf_file')
# print(text)

In [4]:
with open('resume.txt', 'w') as file:
    file.write(extracted_text)

In [5]:
with open('resume.txt', 'r') as file:
    extracted_text = file.read()


# Extraction of features

In [7]:
import google.generativeai as genai
from pathlib import Path

# Replace 'YOUR_API_KEY' with your actual API key
genai.api_key = 'YOUR_API_KEY'

# Use the generative model 'gemini-pro' for extraction
model = genai.GenerativeModel('gemini-pro')

def process_text_file_and_generate(prompt, file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            file_content = file.read()

            generated_content = model.generate_content(prompt + "\n" + file_content)
            print("Generated Content:", generated_content.text)

    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error processing the file: {e}")

# Merge multiple prompt results into a json file

In [8]:
file_path = 'resume.txt'
prompts = {
    "Name": "What is the name present in resume",
    "Email": "What is the email present in resume",
    "Phone No": "What is the mobile no present in resume",
    "Education": "What are the educational qualifications present in resume? Answer in points and keep it precise.",
    "Skills": "What are the skills present in resume? Answer in points",
    "Work Experience": "What is/are the work experience present in resume?  Answer in points and keep it precise."
}


In [11]:
generated_content_dict = {}

def process_text_file_and_generate(prompt, file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            file_content = file.read()

            generated_content = model.generate_content(prompt + "\n" + file_content)
            return generated_content.text

    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    except Exception as e:
        print(f"Error processing the file: {e}")
        return None

for key, prompt in prompts.items():
    generated_content = process_text_file_and_generate(prompt, file_path)

    generated_content_dict[key] = generated_content

for key, value in generated_content_dict.items():
    print(f"{key}: {value}")


Name: Dikesh Ray
Email: Dikesh.ray@gmail.com
Phone No: 7827137507
Education: - MCA from UPTU with 70%.
- BCA from JAMIA HAMDARD University with 71%, New delhi.
- 10+2 from Bihar intermediate examination council, Patna.
- 10th from Bihar school examination board, Patna.
Skills: Skills present in resume:

* Programming Languages: C#, Java Script, HTML
* Web Technologies: ASP.Net, WCF, JQuery, Json, Ajax, AngularJS, MVC, Ms Azure
* Databases: SQL Server
* IDE Tools: Visual Studio
* Strong analytical and leadership skills
* Problem-solving and decision-making skills
* Ability to learn new things
* Team player
Work Experience: - Working with BLS international Services Itd. as a Software Developer from 05th June 2017 to till date.
- Working with BRAWN SOFTECH as a Software Developer from 20th September 2016 to 31st March 2017.
- Working with HYBEC IT SOLUTIONS LIMITED as a Software Developer from 01st June 2015 to 31st August 2016.
- Six months Industrial Training in Microsoft .Net at Swarn

In [12]:
import json
with open('extracted_features.json', 'w') as f:
    json.dump(generated_content_dict, f, indent=4)
print("Extraction completed. Extracted features saved in 'extracted_features.json'.")

Extraction completed. Extracted features saved in 'extracted_features.json'.
